## This tests the QED-RHF gradient components against Hilbert's and against numerical components


In [1]:
from oo_cqed_rhf import CQEDRHFCalculator
import numpy as np
import psi4
psi4.core.be_quiet()

In [2]:
# Expected gradient contributions for Water in a STO-3G basis set with the following geometry:
h2o_string = """
         O            0.000000000000     0.000000000000    -0.068516219320
         H            0.000000000000    -0.790689573744     0.543701060715
         H            0.000000000000     0.790689573744     0.543701060715
no_reorient
nocom
symmetry c1
"""

# no cavity
lambda_vector = np.array([0, 0, 0.05])
print(lambda_vector)
# options for H2O
psi4_options = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}





[0.   0.   0.05]


### The next block will instantiate the class and compute the CQED-RHF energy

In [3]:
calc = CQEDRHFCalculator(lambda_vector, h2o_string, psi4_options)
calc.calc_cqed_rhf_energy()
_expected_qed_rhf_energy = -76.016355284146
print(F"CQED-RHF ENERGY Matches Expected Value: {np.isclose(calc.cqed_rhf_energy, _expected_qed_rhf_energy, 1e-9,1e-9)}")

CQED-RHF ENERGY Matches Expected Value: True


### The next block will run several gradient calculations and print summaries of the results:
1. `self.compute_analytic_gradient(use_psi4=False)` will compute all CQED-RHF gradient terms using class methods.  The following attributes will be assigned:

   `.overlap_gradient` - the Pulay force

   `.kinetic_gradient` - the kinetic gradient term

   `.potential_gradient` - the potential gradient term

   `.J_gradient` - the J gradient term

   `.K_gradient` - the K gradient term

   `.nuclear_repulsion_gradient` - the nuclear repulsion gradient term

   `.canonical_scf_gradient` - the sum of all of the above gradient terms, i.e. all of the terms that contribute to the RHF gradient

   `.o_dse_gradient` - the one-electron dipole self energy gradient term (from quadrupole)

   `.K_dse_gradient` - the two-electron dipole self energy gradient term (from exchange dipole-dipole term

   `.qedrhf_gradient` - the total qed-rhf gradient, which is the sum of the canonical_scf_gradient and the dse gradient terms

2. `self.compute_numerical_gradient()` will compute the CQED-RHF gradient using centered finite differences

3. `self.compute_analytic_gradient(use_psi4=True)` will compute the canonical RHF gradient terms using `psi4.core.scfgrad(wfn)`.  Importantly, this will first update the `wfn` object using CQED-RHF quantities, so it should give the same gradient as the `.canonical_scf_gradient` computed in step 1.


Following these three computations, the following information will be printed:

1. The total analytical CQED-RHF gradient computed in step 1 above
2. The numerical CQED-RHF gradient computed in step 2 above
3. The norm of the difference between the analytical and numerical gradients
4. The analytical canonical RHF gradient computed in step 1 above
5. The analytical canonical RHF gradient computed from psi4.core.scfgrad() in step 3 above
6. The norm of the difference between our analytical and psi4's analytical canonical RHF gradient

**Note** It seems that psi4's scfgrad may utilize density fitting, so the agreement between our canonical RHF gradient and psi4s may be on the order of the density fitting error, 1e-6 or 1e-5.
   

In [4]:
calc.gradient_summary()

Time for Fock matrix term: 5.060e-03 s
Time for One-electron gradient terms: 5.122e-03 s
Time for Two-electron gradient terms: 4.061e-01 s
Time for Nuclear repulsion gradient: 2.429e-04 s
Time for Dipole-dipole gradient: 1.060e-03 s
Time for Quadrupole gradient: 2.401e-04 s
Final gradient assembly using class: 4.181e-01 s
Time to compute numerical CQED-RHF gradient: 1.9457 seconds

Analytical CQED-RHF Gradient:

[[-3.76017513e-17 -1.40426403e-16 -6.18354868e-02]
 [-1.51104989e-17 -4.35899496e-02  3.09177434e-02]
 [ 5.27122502e-17  4.35899496e-02  3.09177434e-02]]

Numerical CQED-RHF Gradient:

[[ 3.76003023e-09 -2.25601814e-09 -6.18354847e-02]
 [ 6.76805442e-09 -4.35899478e-02  3.09177412e-02]
 [ 3.76003023e-10  4.35899447e-02  3.09177435e-02]]

Difference between analytical and numerical gradient: 1.0065e-08

Using Psi4 to compute the canonical RHF gradient for comparison...

Time for SCF gradient via Psi4: 1.306e-02 s
Time for Quadrupole gradient: 3.109e-04 s
Time Dipole-dipole gradi